In [3]:
import numpy as np
# from torchvision import datasets, transforms
# import torchvision
# import torch
# import torch.optim as optim
# from torch.optim import lr_scheduler
# import torch.nn as nn
import os
# from PIL import Image
# from torch.autograd import Variable
from keras import applications

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=3)

Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- Tunnel connection failed: 407 Proxy Authentication Required ( Forefront TMG requires authorization to fulfill the request. Access to the Web Proxy filter is denied.  )

In [ ]:
# model = torchvision.models.inception_v3(pretrained=True)   # import the pre-trained inception v3 model

In [16]:
for param in model.parameters():
    param.requires_grad = False      # only the final layer is to be trained, hence their gradient is not required

In [17]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,3)   # new no. of classes is 2 

In [34]:
use_gpu = torch.cuda.is_available()
print use_gpu
if use_gpu:
    model = model.cuda()       # using GPU for processing

True


In [37]:
class Preprocess(object):    # the images in the train set are read in PIL format and the resized and converted to Tensors
    """Resize and convert PIL images to to Tensors."""

    def __call__(self, sample):
        image = sample

        image = image.resize((299,299),Image.NEAREST)
        image = np.array(image,dtype='float32')
        image = image.transpose()
        image = torch.from_numpy(image)
        return image


In [39]:
data_transform = transforms.Compose([    # the transform class to apply to each of the image being read
        Preprocess()
])

In [40]:
tiny_dataset = datasets.ImageFolder(root='~/workspace/transfer_learning/train_image',transform=data_transform)  
img_dataset = {'train':tiny_dataset}
dataset_loader = torch.utils.data.DataLoader(tiny_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)
dataloaders = {'train':dataset_loader}

In [41]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data
            
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
            
                _, preds = torch.max(outputs[0], 1)
                loss = criterion(outputs[0], labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.Adam(model.fc.parameters(), lr=0.001) #momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)


In [44]:
model = train_model(model, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------


Process Process-24:
KeyboardInterrupt
Process Process-23:
Traceback (most recent call last):
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-21:


KeyboardInterrupt: 

Traceback (most recent call last):
    self.run()
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-22:
Traceback (most recent call last):
    self.run()
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/siplab/anaconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    self.run()
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/siplab/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, i